In [3]:
from zmax_datasets.datasets.usleep import USleepDataset
from pathlib import Path
import numpy as np

DATASETS_DIR = Path("/project/4180000.46/zmax_datasets/accelerometer_usleep")
DATASET_NAME = "donders_2018"

dataset = USleepDataset(data_dir=DATASETS_DIR / DATASET_NAME)
print(dataset.n_recordings)
print(dataset.recording_ids)

6
['P8-night3', 'P11-night3', 'P13-night3', 'P18-night3', 'P20-night3', 'P21-night2']


In [4]:
# Get a sample recording
sample_recording = dataset.get_recording("P8-night3")
print("Available data types:")
print(sample_recording.data_types)

Available data types:
{'ACCELEROMETER_X_dynamic': DataType(channel='ACCELEROMETER_X_dynamic', sampling_rate=64.0), 'ACCELEROMETER_Y_dynamic': DataType(channel='ACCELEROMETER_Y_dynamic', sampling_rate=64.0), 'ACCELEROMETER_Z_dynamic': DataType(channel='ACCELEROMETER_Z_dynamic', sampling_rate=64.0), 'ACCELEROMETER_magnitude': DataType(channel='ACCELEROMETER_magnitude', sampling_rate=64.0), 'ACCELEROMETER_magnitude_derivative': DataType(channel='ACCELEROMETER_magnitude_derivative', sampling_rate=64.0), 'NOISE': DataType(channel='NOISE', sampling_rate=64.0)}


In [ ]:
# Load all accelerometer signals
acc_x_dynamic = sample_recording.read_data_type("ACCELEROMETER_X_dynamic")
acc_y_dynamic = sample_recording.read_data_type("ACCELEROMETER_Y_dynamic")
acc_z_dynamic = sample_recording.read_data_type("ACCELEROMETER_Z_dynamic")
acc_x_gravity = sample_recording.read_data_type("ACCELEROMETER_X_gravity")
acc_y_gravity = sample_recording.read_data_type("ACCELEROMETER_Y_gravity")
acc_z_gravity = sample_recording.read_data_type("ACCELEROMETER_Z_gravity")
acc_mag = sample_recording.read_data_type("ACCELEROMETER_magnitude")
acc_mag_deriv = sample_recording.read_data_type("ACCELEROMETER_magnitude_derivative")
acc_pitch = sample_recording.read_data_type("ACCELEROMETER_pitch")
acc_roll = sample_recording.read_data_type("ACCELEROMETER_roll")

# Create time axis in seconds for the signals
duration_seconds = len(acc_x_dynamic.array.squeeze()) / acc_x_dynamic.sample_rate
time = np.linspace(0, duration_seconds, len(acc_x_dynamic.array.squeeze()))
print(f"\nSignal duration: {duration_seconds:.1f} seconds ({duration_seconds/60:.1f} minutes)")
print(f"Sampling rate: {acc_x_dynamic.sample_rate} Hz")


2025-09-05 13:31:15.386 | INFO     | zmax_datasets.datasets.base:read_data_type:36 - Reading data type: ACCELEROMETER_X_dynamic
2025-09-05 13:31:15.613 | INFO     | zmax_datasets.datasets.base:read_data_type:36 - Reading data type: ACCELEROMETER_Y_dynamic
2025-09-05 13:31:15.820 | INFO     | zmax_datasets.datasets.base:read_data_type:36 - Reading data type: ACCELEROMETER_Z_dynamic
2025-09-05 13:31:15.918 | INFO     | zmax_datasets.datasets.base:read_data_type:36 - Reading data type: ACCELEROMETER_X_gravity


MissingDataTypeError: Data type ACCELEROMETER_X_gravity not found in P8-night3

In [8]:
import ipywidgets as widgets
from IPython.display import display
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create widgets for controlling the window
start_slider = widgets.FloatSlider(
    value=0,
    min=0,
    max=duration_seconds - 60,  # Leave room for the window
    step=10,
    description='Start Time (s):',
    style={'description_width': 'initial'},
    layout={'width': '500px'}
)

window_size = widgets.Dropdown(
    options=[('10 seconds', 10), ('30 seconds', 30), ('1 minute', 60), ('5 minutes', 300)],
    value=60,
    description='Window Size:',
    style={'description_width': 'initial'}
)

# Create initial figure
fig = go.FigureWidget(make_subplots(
    rows=6, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    row_heights=[0.2, 0.2, 0.15, 0.15, 0.15, 0.15],
    subplot_titles=("Dynamic Acceleration Components", "Gravity Components", 
                   "Pitch", "Roll", "Magnitude", "Magnitude Derivative")
))

# Initialize with empty traces
# Add dynamic acceleration components to first subplot
fig.add_trace(go.Scatter(name="X Dynamic", line=dict(color='red')), row=1, col=1)
fig.add_trace(go.Scatter(name="Y Dynamic", line=dict(color='green')), row=1, col=1)
fig.add_trace(go.Scatter(name="Z Dynamic", line=dict(color='blue')), row=1, col=1)

# Add gravity components to second subplot
fig.add_trace(go.Scatter(name="X Gravity", line=dict(color='red', dash='dot')), row=2, col=1)
fig.add_trace(go.Scatter(name="Y Gravity", line=dict(color='green', dash='dot')), row=2, col=1)
fig.add_trace(go.Scatter(name="Z Gravity", line=dict(color='blue', dash='dot')), row=2, col=1)

# Add pitch and roll
fig.add_trace(go.Scatter(name="Pitch", line=dict(color='cyan')), row=3, col=1)
fig.add_trace(go.Scatter(name="Roll", line=dict(color='magenta')), row=4, col=1)

# Add magnitude and derivative to their own subplots
fig.add_trace(go.Scatter(name="Magnitude", line=dict(color='purple')), row=5, col=1)
fig.add_trace(go.Scatter(name="Magnitude Derivative", line=dict(color='orange')), row=6, col=1)

# Update layout
fig.update_layout(
    height=1200,  # Increased height to accommodate new subplots
    title=f"Accelerometer Signal Analysis - Recording {str(sample_recording)}",
    showlegend=True,
    template="plotly_white"
)

# Update axes labels and ranges
fig.update_yaxes(title_text="Dynamic Acceleration (g)", row=1, col=1)
fig.update_yaxes(title_text="Gravity (g)", row=2, col=1)
fig.update_yaxes(title_text="Pitch (degrees)", range=[-np.pi, np.pi], row=3, col=1)
fig.update_yaxes(title_text="Roll (degrees)", range=[-np.pi, np.pi], row=4, col=1)
fig.update_yaxes(title_text="Magnitude (g)", row=5, col=1)
fig.update_yaxes(title_text="Magnitude Derivative", row=6, col=1)
fig.update_xaxes(title_text="Time (seconds)", row=6, col=1)

def update_plot(start_time, window_duration):
    # Calculate indices for the window
    start_idx = int(start_time * acc_x_dynamic.sample_rate)
    end_idx = int((start_time + window_duration) * acc_x_dynamic.sample_rate)
    
    # Create time array for the window
    time_window = np.linspace(start_time, start_time + window_duration, end_idx - start_idx)
    
    # Get signal segments
    # Dynamic components
    x_dynamic = acc_x_dynamic.array.squeeze()[start_idx:end_idx]
    y_dynamic = acc_y_dynamic.array.squeeze()[start_idx:end_idx]
    z_dynamic = acc_z_dynamic.array.squeeze()[start_idx:end_idx]
    
    # Gravity components
    x_gravity = acc_x_gravity.array.squeeze()[start_idx:end_idx]
    y_gravity = acc_y_gravity.array.squeeze()[start_idx:end_idx]
    z_gravity = acc_z_gravity.array.squeeze()[start_idx:end_idx]
    
    # Pitch and roll
    pitch_signal = acc_pitch.array.squeeze()[start_idx:end_idx]
    roll_signal = acc_roll.array.squeeze()[start_idx:end_idx]
    
    # Magnitude and derivative
    mag_signal = acc_mag.array.squeeze()[start_idx:end_idx]
    mag_deriv_signal = acc_mag_deriv.array.squeeze()[start_idx:end_idx]
    
    # Update traces with new data
    with fig.batch_update():
        # Update dynamic components
        fig.data[0].x = time_window
        fig.data[0].y = x_dynamic
        
        fig.data[1].x = time_window
        fig.data[1].y = y_dynamic
        
        fig.data[2].x = time_window
        fig.data[2].y = z_dynamic
        
        # Update gravity components
        fig.data[3].x = time_window
        fig.data[3].y = x_gravity
        
        fig.data[4].x = time_window
        fig.data[4].y = y_gravity
        
        fig.data[5].x = time_window
        fig.data[5].y = z_gravity
        
        # Update pitch and roll
        fig.data[6].x = time_window
        fig.data[6].y = pitch_signal
        
        fig.data[7].x = time_window
        fig.data[7].y = roll_signal
        
        # Update magnitude
        fig.data[8].x = time_window
        fig.data[8].y = mag_signal
        
        # Update magnitude derivative
        fig.data[9].x = time_window
        fig.data[9].y = mag_deriv_signal

# Create the interactive plot
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        update_plot(start_slider.value, window_size.value)

# Link the widgets to the update function
start_slider.observe(on_change)
window_size.observe(on_change)

# Display widgets and initial plot
display(widgets.HBox([start_slider, window_size]))
display(fig)

# Initialize the plot
update_plot(start_slider.value, window_size.value)


FigureWidget({
    'data': [{'line': {'color': 'red'},
              'name': 'X Dynamic',
              'type': 'scatter',
              'uid': '36225bb7-b3b1-4da5-b72d-67f1c274bb40',
              'xaxis': 'x',
              'yaxis': 'y'},
             {'line': {'color': 'green'},
              'name': 'Y Dynamic',
              'type': 'scatter',
              'uid': 'fbd889c4-ef32-430b-b05d-48489f0aecfe',
              'xaxis': 'x',
              'yaxis': 'y'},
             {'line': {'color': 'blue'},
              'name': 'Z Dynamic',
              'type': 'scatter',
              'uid': '99657177-bb70-4add-a507-8a446bec843b',
              'xaxis': 'x',
              'yaxis': 'y'},
             {'line': {'color': 'red', 'dash': 'dot'},
              'name': 'X Gravity',
              'type': 'scatter',
              'uid': 'f7312a04-797a-4a99-bc1c-bfc2eef514ad',
              'xaxis': 'x2',
              'yaxis': 'y2'},
             {'line': {'color': 'green', 'dash': 'dot'},
    